# Neural Nets from Scratch in Julia

## Lesson 9: Inversion and division

* In this video we'll implement invesion and division for *Values*.
* [Documentation site here](https://mikesaint-antoine.github.io/SimpleGrad.jl)
* [Github repo here](https://github.com/mikesaint-antoine/SimpleGrad.jl)

In [1]:
## code so far

mutable struct Value{opType} <: Number
    data::Float64
    grad::Float64
    op::opType
end


struct Operation{FuncType, ArgTypes}
    op::FuncType
    args::ArgTypes
end

# constructor -- Value(data, grad, op)
Value(x::Number) = Value(Float64(x), 0.0, nothing);


import Base.show
function show(io::IO, value::Value)
    print(io, "Value(",value.data,")")
end


import Base.==
function ==(a::Value, b::Value)
     return a===b
end


import Base.+
function +(a::Value, b::Value)

    out = a.data + b.data    
    result = Value(out, 0.0, Operation(+, (a,b))) # Value(data, grad, op)
    return result # this should be a Value
 
end



backprop!(val::Value{Nothing}) = nothing


function backprop!(val::Value{Operation{FunType, ArgTypes}}) where {FunType<:typeof(+), ArgTypes}
    
    # val = a + b
    # update a.grad, b.grad
    
    val.op.args[1].grad += val.grad
    val.op.args[2].grad += val.grad
    
end




function backward(a::Value)
    
    
    function build_topo(v::Value, visited=Value[], topo=Value[])
    
        if !(v in visited)
            
            push!(visited, v)
            
            if v.op != nothing
                for operand in v.op.args
                    
                    if operand isa Value
                        build_topo(operand, visited, topo)
                    end
                end 
            end
            
            push!(topo, v) 
            
            
        end
        return topo
    end
    
    
    
    topo = build_topo(a)
    
    a.grad = 1
    #da/da = 1
    
    for node in reverse(topo)
        backprop!(node)
    end
    
    
end


Base.promote_rule(::Type{<:Value}, ::Type{T}) where {T<:Number} = Value




import Base.*
function *(a::Value, b::Value)

    out = a.data * b.data    
    result = Value(out, 0.0, Operation(*, (a,b))) # Value(data, grad, op)
    return result # this should be a Value
 
end



function backprop!(val::Value{Operation{FunType, ArgTypes}}) where {FunType<:typeof(*), ArgTypes}
    
    # val = a * b
    # update a.grad, b.grad
    
    val.op.args[1].grad += val.op.args[2].data * val.grad    
    val.op.args[2].grad += val.op.args[1].data * val.grad
    
end



import Base.-

# negation
function -(a::Value)
    
    return a * -1
    
end

# subtraction
function -(a::Value, b::Value)
    
    return a + (-b)
    
    
end

- (generic function with 199 methods)

In [7]:
import Base.inv
function inv(a::Value)
    
    out = 1.0 / a.data
    result = Value(out, 0.0, Operation(inv, (a,))) # Value(data, grad, op)
    return result # this should be a Value    
    
    
end


function backprop!(val::Value{Operation{FunType, ArgTypes}}) where {FunType<:typeof(inv), ArgTypes}
    
    # val = inv(a)
    # update a.grad
    
    # a.grad -= (1.0 / a.data^2) * val.grad
    
    val.op.args[1].grad -= (1.0 / val.op.args[1].data^2) * val.grad
    
    
end


# y = inv(x) = 1/x
# y = x ^ (-1)

# dy/dx = (-1) * x^(-2)
# dy/dx = -(1 / x^2)


backprop! (generic function with 4 methods)

In [8]:
# inversion 

x = Value(2)

y = inv(x)


backward(y)


println(x.grad)
# dy/dx = -(1/x^2) = -1/4 = -0.25



-0.25


In [9]:
import Base./
function /(a::Value, b::Value)
     
    # a/b = a * b^(-1)
    
    return a * inv(b)
    
    
end

/ (generic function with 107 methods)

In [10]:
x = Value(8)
y = Value(2)

z = x / y


backward(z)

println(x.grad)
# dz/dx = 1/y = 1/2 = 0.5

println(y.grad)
# = x * y^(-1)
# dz/dy = -x *y^(-2) = -x / y^2 = -8/4 = -2

0.5
-2.0
